# Task III - Game Simulation and Finding Nash Equilibrium

This notebook implements iterative game theory simulation to find the Nash Equilibrium where no seller has incentive to change their strategy.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1. Load Seller Configuration and Historical Data

In [ ]:
seller_config_df = pd.read_csv('seller_config.csv')
initial_profit_df = pd.read_csv('initial_profit_summary.csv')
profit_data_df = pd.read_csv('initial_profit_calculations.csv')

print("Seller Configuration:")
print(seller_config_df.to_string(index=False))

print("\n\nInitial Profit Summary (from Task II):")
print(initial_profit_df.to_string(index=False))

print("\n\nInitial Profit Data Sample:")
print(profit_data_df.head())

## 2. Define Updated Seller Class with Dynamic Strategy

In [ ]:
class DynamicSeller:
    def __init__(self, name, price, cost, advertising_budget, base_demand, items_list, initial_profit):
        self.name = name
        self.price = price
        self.cost = cost
        self.advertising_budget = advertising_budget
        self.base_demand = base_demand
        self.items = items_list
        self.profit = 0
        self.demand = 0
        self.initial_profit = initial_profit
        self.price_history = [price]
        self.ad_history = [advertising_budget]
        self.profit_history = [initial_profit]
        self.demand_history = [0]
    
    def update_strategy(self, new_price, new_ad_budget):
        self.price = new_price
        self.advertising_budget = new_ad_budget
        self.price_history.append(new_price)
        self.ad_history.append(new_ad_budget)
    
    def record_metrics(self, profit, demand):
        self.profit = profit
        self.demand = demand
        self.profit_history.append(profit)
        self.demand_history.append(demand)
    
    def __repr__(self):
        return f"DynamicSeller({self.name}, p={self.price:.2f}, m={self.advertising_budget:.2f})"

sellers = [
    DynamicSeller('Seller_A', seller_config_df.iloc[0]['Price'], 
                  seller_config_df.iloc[0]['Cost'], 
                  seller_config_df.iloc[0]['Advertising_Budget'],
                  seller_config_df.iloc[0]['Base_Demand'],
                  [],
                  initial_profit_df[initial_profit_df['Seller'] == 'Seller_A']['Avg_Initial_Profit'].values[0]),
    DynamicSeller('Seller_B', seller_config_df.iloc[1]['Price'], 
                  seller_config_df.iloc[1]['Cost'], 
                  seller_config_df.iloc[1]['Advertising_Budget'],
                  seller_config_df.iloc[1]['Base_Demand'],
                  [],
                  initial_profit_df[initial_profit_df['Seller'] == 'Seller_B']['Avg_Initial_Profit'].values[0]),
    DynamicSeller('Seller_C', seller_config_df.iloc[2]['Price'], 
                  seller_config_df.iloc[2]['Cost'], 
                  seller_config_df.iloc[2]['Advertising_Budget'],
                  seller_config_df.iloc[2]['Base_Demand'],
                  [],
                  initial_profit_df[initial_profit_df['Seller'] == 'Seller_C']['Avg_Initial_Profit'].values[0])
]

print("Dynamic Sellers Initialized (with Initial Profits Loaded):")
for seller in sellers:
    print(f"  {seller}")
    print(f"    Initial Profit: ${seller.initial_profit:.2f}")

## 3. Define Demand and Profit Functions

In [ ]:
alpha = 0.05
beta = 15.0
gamma = 0.8
influence_score = 100

def calculate_demand(seller_i, seller_j, alpha, beta, gamma, influence_score):
    D_i = (seller_i.base_demand + 
           alpha * seller_i.advertising_budget + 
           beta * (seller_i.price - seller_j.price) + 
           gamma * influence_score)
    return max(D_i, 0)

def calculate_profit(seller_i, seller_j, alpha, beta, gamma, influence_score):
    D_i = calculate_demand(seller_i, seller_j, alpha, beta, gamma, influence_score)
    profit = (seller_i.price - seller_i.cost) * D_i - seller_i.advertising_budget
    return profit, D_i

print("Game Parameters:")
print(f"  α (advertising sensitivity): {alpha}")
print(f"  β (price sensitivity): {beta}")
print(f"  γ (social influence): {gamma}")
print(f"  Initial influence score: {influence_score}")

## 4. Define Best Response Functions

In [ ]:
def find_best_response_price(seller_i, seller_j, alpha, beta, gamma, influence_score, 
                            price_range=(0.1, 15), step=0.2, constraint=True):
    best_profit = float('-inf')
    best_price = seller_i.price
    
    for test_price in np.arange(price_range[0], price_range[1], step):
        if constraint and test_price <= seller_i.cost:
            continue
        
        temp_seller = DynamicSeller(
            seller_i.name, test_price, seller_i.cost, 
            seller_i.advertising_budget, seller_i.base_demand, seller_i.items, seller_i.initial_profit
        )
        profit, _ = calculate_profit(temp_seller, seller_j, alpha, beta, gamma, influence_score)
        
        if profit > best_profit:
            best_profit = profit
            best_price = test_price
    
    return best_price, best_profit

def find_best_response_advertising(seller_i, seller_j, alpha, beta, gamma, influence_score,
                                    ad_range=(50, 300), step=10):
    best_profit = float('-inf')
    best_ad = seller_i.advertising_budget
    
    for test_ad in np.arange(ad_range[0], ad_range[1], step):
        temp_seller = DynamicSeller(
            seller_i.name, seller_i.price, seller_i.cost,
            test_ad, seller_i.base_demand, seller_i.items, seller_i.initial_profit
        )
        profit, _ = calculate_profit(temp_seller, seller_j, alpha, beta, gamma, influence_score)
        
        if profit > best_profit:
            best_profit = profit
            best_ad = test_ad
    
    return best_ad, best_profit

print("Best Response Functions Defined")
print("  - find_best_response_price(): Finds optimal price")
print("  - find_best_response_advertising(): Finds optimal ad budget")

## 5. Implement Iterative Game Simulation

In [ ]:
def run_game_simulation(sellers, iterations=50, alpha=0.05, beta=15.0, 
                          gamma=0.8, influence_score=100, 
                          learning_rate=0.6, enable_both_strategies=True):
    
    convergence_history = []
    
    for iteration in range(iterations):
        iteration_data = {'Iteration': iteration}
        max_change = 0
        
        for i, seller_i in enumerate(sellers):
            for j, seller_j in enumerate(sellers):
                if i != j:
                    old_price = seller_i.price
                    old_ad = seller_i.advertising_budget
                    
                    if enable_both_strategies:
                        best_price, _ = find_best_response_price(
                            seller_i, seller_j, alpha, beta, gamma, influence_score,
                            price_range=(max(0.1, seller_i.cost - 1), 15), step=0.3
                        )
                        
                        best_ad, _ = find_best_response_advertising(
                            seller_i, seller_j, alpha, beta, gamma, influence_score,
                            ad_range=(50, 300), step=15
                        )
                        
                        new_price = old_price + learning_rate * (best_price - old_price)
                        new_ad = old_ad + learning_rate * (best_ad - old_ad)
                    else:
                        if iteration % 2 == 0:
                            best_price, _ = find_best_response_price(
                                seller_i, seller_j, alpha, beta, gamma, influence_score,
                                price_range=(max(0.1, seller_i.cost - 1), 15), step=0.3
                            )
                            new_price = old_price + learning_rate * (best_price - old_price)
                            new_ad = old_ad
                        else:
                            best_ad, _ = find_best_response_advertising(
                                seller_i, seller_j, alpha, beta, gamma, influence_score,
                                ad_range=(50, 300), step=15
                            )
                            new_price = old_price
                            new_ad = old_ad + learning_rate * (best_ad - old_ad)
                    
                    new_price = max(seller_i.cost * 1.01, new_price)
                    new_ad = max(50, new_ad)
                    
                    seller_i.update_strategy(new_price, new_ad)
                    
                    price_change = abs(new_price - old_price)
                    ad_change = abs(new_ad - old_ad)
                    change = max(price_change, ad_change)
                    max_change = max(max_change, change)
        
        for i, seller_i in enumerate(sellers):
            for j, seller_j in enumerate(sellers):
                if i != j:
                    profit, demand = calculate_profit(seller_i, seller_j, alpha, beta, 
                                                     gamma, influence_score)
                    seller_i.record_metrics(profit, demand)
        
        for seller in sellers:
            iteration_data[f"{seller.name}_Price"] = seller.price
            iteration_data[f"{seller.name}_AD"] = seller.advertising_budget
            iteration_data[f"{seller.name}_Profit"] = seller.profit
            iteration_data[f"{seller.name}_Demand"] = seller.demand
        
        iteration_data['Max_Change'] = max_change
        convergence_history.append(iteration_data)
        
        if (iteration + 1) % 10 == 0 or iteration == 0:
            print(f"Iteration {iteration}: Max Change = {max_change:.6f}")
        
        if max_change < 0.001:
            print(f"\nConverged at iteration {iteration}")
            break
    
    return pd.DataFrame(convergence_history)

print("Game Simulation Function Defined")
print("  - Iteratively updates prices and advertising")
print("  - Tracks convergence to Nash Equilibrium")
print("  - Uses learning rate to smooth updates")

## 6. Run Game Simulation

In [ ]:
print("Starting Game Simulation...")
print("="*70)

simulation_results = run_game_simulation(
    sellers, 
    iterations=100,
    alpha=alpha,
    beta=beta,
    gamma=gamma,
    influence_score=influence_score,
    learning_rate=0.6,
    enable_both_strategies=True
)

print("\n" + "="*70)
print(f"Simulation completed with {len(simulation_results)} iterations")

## 7. Analyze Nash Equilibrium Results

In [ ]:
print("\nNASH EQUILIBRIUM ANALYSIS")
print("="*70)

print("\nFinal Equilibrium State:")
print("-" * 70)

for seller in sellers:
    print(f"\n{seller.name}:")
    print(f"  Final Price: ${seller.price:.4f}")
    print(f"  Final Ad Budget: ${seller.advertising_budget:.2f}")
    print(f"  Final Demand: {seller.demand:.2f} units")
    print(f"  Final Profit: ${seller.profit:.2f}")
    print(f"  Production Cost: ${seller.cost:.2f}")
    print(f"  Margin per Unit: ${seller.price - seller.cost:.4f}")

print("\n" + "="*70)
print("\nComparison: Initial vs Final (FIXED)")
print("="*70)

for seller in sellers:
    print(f"\n{seller.name}:")
    print(f"  Price Change: ${seller.price_history[0]:.4f} -> ${seller.price:.4f} " + 
          f"({((seller.price - seller.price_history[0])/seller.price_history[0]*100):+.2f}%)")
    print(f"  Ad Budget Change: ${seller.ad_history[0]:.2f} -> ${seller.advertising_budget:.2f} " + 
          f"({((seller.advertising_budget - seller.ad_history[0])/seller.ad_history[0]*100):+.2f}%)")
    if seller.initial_profit != 0:
        profit_change_pct = ((seller.profit - seller.initial_profit)/abs(seller.initial_profit)*100)
    else:
        profit_change_pct = ((seller.profit - seller.initial_profit) / 1) * 100 if seller.profit != seller.initial_profit else 0
    print(f"  Profit Change: ${seller.initial_profit:.2f} -> ${seller.profit:.2f} " + 
          f"({profit_change_pct:+.2f}%)")

## 8. Convergence Analysis

In [ ]:
print("Convergence Metrics:")
print("="*70)

print(f"\nTotal Iterations to Convergence: {len(simulation_results)}")
print(f"Final Max Change: {simulation_results['Max_Change'].iloc[-1]:.8f}")

print(f"\nPrice Stability (std of last 10 iterations):")
last_10_count = min(10, len(simulation_results))
last_iterations = simulation_results.tail(last_10_count)
for seller in sellers:
    prices = last_iterations[f"{seller.name}_Price"].values
    if len(prices) > 1:
        std_price = np.std(prices)
        print(f"  {seller.name}: {std_price:.8f}")

print(f"\nAdvertising Budget Stability (std of last 10 iterations):")
for seller in sellers:
    ads = last_iterations[f"{seller.name}_AD"].values
    if len(ads) > 1:
        std_ad = np.std(ads)
        print(f"  {seller.name}: {std_ad:.4f}")

print(f"\nProfit Convergence:")
for seller in sellers:
    profits_early = simulation_results[f"{seller.name}_Profit"].iloc[:min(5, len(simulation_results))].mean()
    profits_late = simulation_results[f"{seller.name}_Profit"].iloc[-5:].mean()
    if abs(profits_early) > 1:
        improvement = ((profits_late - profits_early) / abs(profits_early)) * 100
    else:
        improvement = 0
    print(f"  {seller.name}: {profits_early:.2f} -> {profits_late:.2f} ({improvement:+.2f}%)")

## 9. Visualization of Convergence

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

ax = axes[0, 0]
for seller in sellers:
    ax.plot(simulation_results['Iteration'], 
            simulation_results[f"{seller.name}_Price"], 
            marker='o', label=seller.name, linewidth=2, markersize=3)
ax.set_xlabel('Iteration')
ax.set_ylabel('Price ($)')
ax.set_title('Price Convergence Over Iterations', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
for seller in sellers:
    ax.plot(simulation_results['Iteration'], 
            simulation_results[f"{seller.name}_AD"], 
            marker='s', label=seller.name, linewidth=2, markersize=3)
ax.set_xlabel('Iteration')
ax.set_ylabel('Advertising Budget ($)')
ax.set_title('Ad Budget Convergence Over Iterations', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 0]
for seller in sellers:
    ax.plot(simulation_results['Iteration'], 
            simulation_results[f"{seller.name}_Profit"], 
            marker='^', label=seller.name, linewidth=2, markersize=3)
ax.set_xlabel('Iteration')
ax.set_ylabel('Profit ($)')
ax.set_title('Profit Evolution Over Iterations', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 1]
ax.semilogy(simulation_results['Iteration'], 
            simulation_results['Max_Change'], 
            marker='d', linewidth=2, markersize=4, color='purple')
ax.axhline(y=0.001, color='r', linestyle='--', label='Convergence Threshold')
ax.set_xlabel('Iteration')
ax.set_ylabel('Max Change (log scale)')
ax.set_title('Convergence Rate', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('task_III_convergence_analysis.png', dpi=100, bbox_inches='tight')
plt.show()

print("Chart saved as 'task_III_convergence_analysis.png'")

## 10. Game Theory Interpretation

In [ ]:
print("\nGAME THEORY INTERPRETATION")
print("="*70)

print("\nNash Equilibrium Definition:")
print("A strategy profile where no player can improve their payoff by")
print("unilaterally changing their strategy, given others' strategies.")

print("\n" + "-"*70)
print("\nEquilibrium Characteristics:")
print("-"*70)

for seller in sellers:
    price_std = np.std(simulation_results[f"{seller.name}_Price"].tail(min(10, len(simulation_results))))
    ad_std = np.std(simulation_results[f"{seller.name}_AD"].tail(min(10, len(simulation_results))))
    price_stable = price_std < 0.01
    ad_stable = ad_std < 5
    
    profits_last = simulation_results[f"{seller.name}_Profit"].tail(min(5, len(simulation_results))).values
    profit_increasing = profits_last[-1] > profits_last[0] if len(profits_last) > 1 else False
    
    print(f"\n{seller.name}:")
    print(f"  Price Stable: {'Yes' if price_stable else 'No'}")
    print(f"  Ad Budget Stable: {'Yes' if ad_stable else 'No'}")
    print(f"  Profit Trend: {'Increasing' if profit_increasing else 'Decreasing/Stable'}")

total_profits = sum([s.profit for s in sellers])
print(f"\nTotal Market Profit: ${total_profits:.2f}")
print(f"Average Profit per Seller: ${total_profits/len(sellers):.2f}")

## 11. Save Equilibrium Results (FIXED)

In [ ]:
equilibrium_results = []
for seller in sellers:
    equilibrium_results.append({
        'Seller': seller.name,
        'Equilibrium_Price': seller.price,
        'Equilibrium_AD_Budget': seller.advertising_budget,
        'Equilibrium_Demand': seller.demand,
        'Equilibrium_Profit': seller.profit,
        'Production_Cost': seller.cost,
        'Profit_Margin': seller.price - seller.cost,
        'Initial_Price': seller.price_history[0],
        'Initial_AD_Budget': seller.ad_history[0],
        'Initial_Profit': seller.initial_profit
    })

equilibrium_df = pd.DataFrame(equilibrium_results)
equilibrium_df.to_csv('nash_equilibrium_results.csv', index=False)

print("Nash Equilibrium Results Saved to 'nash_equilibrium_results.csv'")
print("\nEquilibrium Summary (with Fixed Initial Profits):")
print(equilibrium_df.to_string(index=False))

simulation_results.to_csv('game_simulation_history.csv', index=False)
print("\n\nGame Simulation History Saved to 'game_simulation_history.csv'")